<a href="https://colab.research.google.com/github/jbenasuli/final_project/blob/livia-branch/AmazonDatabase.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
import os
# Find the latest version of spark 2.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.0'
spark_version = 'spark-3.1.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:3 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:7 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:12 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:14 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:15 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic 

In [14]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("BigData-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [22]:
# Selected furniture data
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Furniture_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df_furniture = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df_furniture.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   33670092|R1UUISQ1GKOJTI|B00EE62UAE|     583436067|Trademark Home Po...|            Home|          1|            0|          0|   N|                Y|                 Run|Don't buy it clap...| 2015-08-31|
|         US|   13726692|R1HOJ9WE8VCVOD|B001APXO5C|     465035091|O2-Cool 10-Inch P...|            Home|          5|    

In [23]:
print((df_furniture.count(), len(df_furniture.columns)))

(7013672, 15)


In [24]:
# Drop the non-beneficial columns
columns_to_drop = ["review_id","product_title","helpful_votes","review_headline","review_body","marketplace"]

df_furniture = df_furniture.drop(*columns_to_drop)
df_furniture.show()

+-----------+-----------+----------+--------------+----------------+-----------+-----------+----+-----------------+-----------+
|marketplace|customer_id|product_id|product_parent|product_category|star_rating|total_votes|vine|verified_purchase|review_date|
+-----------+-----------+----------+--------------+----------------+-----------+-----------+----+-----------------+-----------+
|         US|   33670092|B00EE62UAE|     583436067|            Home|          1|          0|   N|                Y| 2015-08-31|
|         US|   13726692|B001APXO5C|     465035091|            Home|          5|          9|   N|                Y| 2015-08-31|
|         US|   50131396|B002HFDLCK|     136507891|            Home|          5|          0|   N|                Y| 2015-08-31|
|         US|   16046884|B00PL9EFPQ|     631238459|            Home|          2|          0|   N|                Y| 2015-08-31|
|         US|   11417282|B00B5TPVQS|     190431573|            Home|          4|          0|   N|       

In [25]:
# Only considering verified purchases
df_furniture = df_furniture.filter(df_furniture.verified_purchase == "Y")
print((df_furniture.count(), len(df_furniture.columns)))

(6263101, 10)


In [26]:
# Selected home data
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Home_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df_home = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df_home.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   33670092|R1UUISQ1GKOJTI|B00EE62UAE|     583436067|Trademark Home Po...|            Home|          1|            0|          0|   N|                Y|                 Run|Don't buy it clap...| 2015-08-31|
|         US|   13726692|R1HOJ9WE8VCVOD|B001APXO5C|     465035091|O2-Cool 10-Inch P...|            Home|          5|    

In [27]:
print((df_home.count(), len(df_home.columns)))

(7013672, 15)


In [29]:
# Drop the non-beneficial columns
df_home = df_home.drop(*columns_to_drop)
df_home.show()

+-----------+-----------+----------+--------------+----------------+-----------+-----------+----+-----------------+-----------+
|marketplace|customer_id|product_id|product_parent|product_category|star_rating|total_votes|vine|verified_purchase|review_date|
+-----------+-----------+----------+--------------+----------------+-----------+-----------+----+-----------------+-----------+
|         US|   33670092|B00EE62UAE|     583436067|            Home|          1|          0|   N|                Y| 2015-08-31|
|         US|   13726692|B001APXO5C|     465035091|            Home|          5|          9|   N|                Y| 2015-08-31|
|         US|   50131396|B002HFDLCK|     136507891|            Home|          5|          0|   N|                Y| 2015-08-31|
|         US|   16046884|B00PL9EFPQ|     631238459|            Home|          2|          0|   N|                Y| 2015-08-31|
|         US|   11417282|B00B5TPVQS|     190431573|            Home|          4|          0|   N|       

In [30]:
# Only considering verified purchases
df_home = df_home.filter(df_home.verified_purchase == "Y")
print((df_home.count(), len(df_home.columns)))

(6263101, 10)


In [31]:
print((df_home.count(), len(df_home.columns)))

(6263101, 10)


In [34]:
# Merge both databases
from functools import reduce  # For Python 3.x
from pyspark.sql import DataFrame

def unionAll(*dfs):
    return reduce(DataFrame.unionAll, dfs)

df_merge = unionAll(df_furniture, df_home)
df_merge.show()


+-----------+-----------+----------+--------------+----------------+-----------+-----------+----+-----------------+-----------+
|marketplace|customer_id|product_id|product_parent|product_category|star_rating|total_votes|vine|verified_purchase|review_date|
+-----------+-----------+----------+--------------+----------------+-----------+-----------+----+-----------------+-----------+
|         US|   33670092|B00EE62UAE|     583436067|            Home|          1|          0|   N|                Y| 2015-08-31|
|         US|   13726692|B001APXO5C|     465035091|            Home|          5|          9|   N|                Y| 2015-08-31|
|         US|   50131396|B002HFDLCK|     136507891|            Home|          5|          0|   N|                Y| 2015-08-31|
|         US|   16046884|B00PL9EFPQ|     631238459|            Home|          2|          0|   N|                Y| 2015-08-31|
|         US|   48013430|B00NQDGHDC|     124050883|            Home|          5|          0|   N|       

In [35]:
print((df_merge.count(), len(df_merge.columns)))

(12526202, 10)


In [36]:
# Get datatypes
df_merge.printSchema()

root
 |-- marketplace: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- product_category: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- review_date: string (nullable = true)



In [39]:
# Transform date column in datetime data type


# For additional data on other products segments:
https://s3.amazonaws.com/amazon-reviews-pds/tsv/index.txt